In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import matplotlib.pyplot as plt
import mplcursors
import numpy as np

# Set up Spotify API credentials
client_id = '8a9262cf15dd4e16bd72737b6e6a93ee'
client_secret = '1b7eb12115884c8eb165f3291eaa55c6'
redirect_uri = 'https://en.wikipedia.org/wiki/India'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_track_id(track_name):
    """Get the Spotify track ID for a given track name."""
    results = sp.search(q='track:' + track_name, type='track')
    return results['tracks']['items'][0]['id']

def get_track_features(track_id):
    """Get the valence and arousal (danceability) for a given track ID."""
    features = sp.audio_features([track_id])
    valence = features[0]['valence']
    arousal = features[0]['danceability']
    return {'valence': valence, 'arousal': arousal}

def get_tracks_data(track_names):
    """Get valence and arousal data for a list of tracks."""
    tracks_data = {}
    for track_name in track_names:
        track_id = get_track_id(track_name)
        track_features = get_track_features(track_id)
        tracks_data[track_name] = track_features
    return tracks_data

track_names = ['Willow', 'Zevia', 'Fall Out Boy', 'Shake it Off']  # Replace with your list of song names
tracks_data = get_tracks_data(track_names)
print(tracks_data)


{'Willow': {'valence': 0.529, 'arousal': 0.392}, 'Zevia': {'valence': 0.372, 'arousal': 0.91}, 'Fall Out Boy': {'valence': 0.282, 'arousal': 0.547}, 'Shake it Off': {'valence': 0.943, 'arousal': 0.647}}


In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error


# Assuming you have a CSV file named 'responses.csv'
df = pd.read_csv('combined_file.csv')
df['video'] = df.iloc[:, 14]
df['Valence_value']=df.iloc[:, 10]
df['Arousal_value']=df.iloc[:, 11]

In [26]:
# Keep only the desired columns
df = df[['Song', 'Valence_value', 'Arousal_value', 'video']]
df = df.dropna(subset=['video'])

print(df)

            Song  Valence_value  Arousal_value video
0         Willow            7.0            5.0    No
1          Zevia            3.0            2.0    No
2   Fall Out Boy            9.0            9.0    No
3   Shake it Off            6.0            8.0    No
4         Willow            7.0            7.0    No
5         Willow            8.0            9.0   Yes
6          Zevia            3.0            1.0   Yes
7   Fall Out Boy            7.0           10.0   Yes
8   Shake it Off            6.0            7.0   Yes
9         Willow            9.0            4.0   Yes
10        Willow            8.0            3.0   Yes


In [38]:
df_willow_yes = df[(df['Song'] == 'Willow') & (df['video'] == 'Yes')]
df_willow_no = df[(df['Song'] == 'Willow') & (df['video'] == 'No')]
euclidean_distances_yes = []
euclidean_distances_no = []
for index, row in df_willow_yes.iterrows():
    x = row['Valence_value']
    y = row['Arousal_value']
    x0 = tracks_data['Willow']['valence']
    y0 = tracks_data['Willow']['arousal']
    euclidean_distance = np.sqrt((x - x0)**2 + (y - y0)**2)
    euclidean_distances_yes.append(euclidean_distance)

# Calculate Euclidean distance for 'no' video
for index, row in df_willow_no.iterrows():
    x = row['Valence_value']
    y = row['Arousal_value']
    x0 = tracks_data['Willow']['valence']
    y0 = tracks_data['Willow']['arousal']
    euclidean_distance = np.sqrt((x - x0)**2 + (y - y0)**2)
    euclidean_distances_no.append(euclidean_distance)

print("Euclidean distances for 'willow' with video 'yes':", euclidean_distances_yes)
print("Euclidean distances for 'willow' with video 'no':", euclidean_distances_no)

Euclidean distances for 'willow' with video 'yes': [11.397960563188487, 9.20736145700819, 7.913122329396911]
Euclidean distances for 'willow' with video 'no': [7.944023225041578, 9.248756943503272]


In [39]:
euclidean_distances = {}

# Iterate over each unique song
for song in df['Song'].unique():
    # Filter the DataFrame for the current song and videos labeled as 'Yes' and 'No'
    df_song_yes = df[(df['Song'] == song) & (df['video'] == 'Yes')]
    df_song_no = df[(df['Song'] == song) & (df['video'] == 'No')]
    
    # Initialize arrays to store Euclidean distances for the current song
    euclidean_distances_yes = []
    euclidean_distances_no = []
    
    # Calculate Euclidean distance for 'Yes' video
    for index, row in df_song_yes.iterrows():
        x = row['Valence_value']
        y = row['Arousal_value']
        x0 = tracks_data[song]['valence']
        y0 = tracks_data[song]['arousal']
        euclidean_distance = np.sqrt((x - x0)**2 + (y - y0)**2)
        euclidean_distances_yes.append(euclidean_distance)
    
    # Calculate Euclidean distance for 'No' video
    for index, row in df_song_no.iterrows():
        x = row['Valence_value']
        y = row['Arousal_value']
        x0 = tracks_data[song]['valence']
        y0 = tracks_data[song]['arousal']
        euclidean_distance = np.sqrt((x - x0)**2 + (y - y0)**2)
        euclidean_distances_no.append(euclidean_distance)
    
    # Store Euclidean distances in the dictionary
    euclidean_distances[song] = {'yes': euclidean_distances_yes, 'no': euclidean_distances_no}

# Print Euclidean distances for each song
for song, distances in euclidean_distances.items():
    print(f"Euclidean distances for '{song}' with video 'Yes':", distances['yes'])
    print(f"Euclidean distances for '{song}' with video 'No':", distances['no'])

Euclidean distances for 'Willow' with video 'Yes': [11.397960563188487, 9.20736145700819, 7.913122329396911]
Euclidean distances for 'Willow' with video 'No': [7.944023225041578, 9.248756943503272]
Euclidean distances for 'Zevia' with video 'Yes': [2.6295406442951212]
Euclidean distances for 'Zevia' with video 'No': [2.8450806666947077]
Euclidean distances for 'Fall Out Boy' with video 'Yes': [11.597013969121534]
Euclidean distances for 'Fall Out Boy' with video 'No': [12.143176396643508]
Euclidean distances for 'Shake it Off' with video 'Yes': [8.119966625547177]
Euclidean distances for 'Shake it Off' with video 'No': [8.924116650963276]
